In [5]:
data_root = "/home/maling/fanqiliang/lung16"
annotation_csv = "/home/maling/fanqiliang/lung16/CSVFILES/annotations.csv"
ct_root = "/home/maling/fanqiliang/lung16/LUNG16"

output_root = "/home/maling/fanqiliang/output_data/medical_data/ct"

In [6]:
import pandas as pd
annotation = pd.read_csv(annotation_csv)


In [7]:
from glob import glob
import os
import math
import SimpleITK as sitk
from PIL import Image
import cv2
import numpy as np


if not os.path.exists(output_root):
    os.makedirs(output_root, exist_ok=True)

annotation_csv = os.path.join(output_root, "annotation.csv")

file_list = []
an_bboxs = []

for ct_idx, (sid, groupobj) in enumerate(annotation.groupby("seriesuid")):
    save_path = os.path.join(output_root, f"{ct_idx}")

    bboxs = []

    mhd_file = glob(os.path.join(ct_root, "**", f"{sid}.mhd"))[0]
    img = sitk.ReadImage(mhd_file)
    arr = sitk.GetArrayFromImage(img)
    x_list = groupobj["coordX"]
    y_list = groupobj["coordY"]
    z_list = groupobj["coordZ"]
    d_list = groupobj["diameter_mm"]

    spacing = img.GetSpacing()  # (s_x, s_y, s_z)
    for x, y, z, d in zip(x_list, y_list, z_list, d_list):
        (i, j, k) = img.TransformPhysicalPointToIndex((x, y, z))
        (dx, dy, dz) = int(d // spacing[0]), int(d // spacing[1]), int(d // spacing[2])
        bboxs.append([(k-math.ceil(dz//2), k+math.ceil(dz//2)), (j-math.ceil(dy//2), j+math.ceil(dy//2)), (i-math.ceil(dx//2), i+math.ceil(dx//2))])  # [(z_min, z_max), (y_min, y_max), (x_min, x_max)]

    for idx, slice in enumerate(arr):
        data_path = os.path.join(save_path, "data")
        label_path = os.path.join(save_path, "label")
        if not os.path.exists(data_path):
            os.makedirs(data_path, exist_ok=True)
        if not os.path.exists(label_path):
            os.makedirs(label_path, exist_ok=True)

        # data
        sitk.WriteImage(sitk.GetImageFromArray(slice), os.path.join(data_path, f"{idx}.dcm"))
        file_list.append(os.path.join(f"{ct_idx}", "data", f"{idx}.dcm"))
        
        # label
        _bbox = []
        jpg = Image.fromarray((slice - slice.min()) / (slice.max() - slice.min()) * 255).convert("RGB")
        jpg = np.asarray(jpg)
        for (z_min, z_max), (y_min, y_max), (x_min, x_max) in bboxs:
            if z_min <= idx <= z_max:
                cv2.rectangle(jpg, (x_min, y_min), (x_max, y_max), color=(0, 255, 0), thickness=1)
                _bbox.append([x_min, y_min, x_max, y_max])
        an_bboxs.append(_bbox)
        cv2.imwrite(os.path.join(label_path, f"{idx}.jpg"), jpg)

    annotation = pd.DataFrame({
        "file": file_list,
        "bbox": an_bboxs
    })
    annotation.to_csv(annotation_csv, index=False)
        

RuntimeError: Exception thrown in SimpleITK ImageFileWriter_Execute: /tmp/SimpleITK-build/ITK/Modules/IO/GDCM/src/itkGDCMImageIO.cxx:1154:
itk::ERROR: itk::ERROR: GDCMImageIO(0x55e7cc833ed0): A Floating point buffer was passed but the stored pixel type was not specified.This is currently not supported